## Суть задания:

Необходимо прочитать .csv "test.csv" в котором записаны геоданные (информация в заголовках) в системе координат `source` (нужно будет с помощью pyproj перевести в `target` для отображения на карте с помощью библиотеки folium). Предвартельно нужно будет преобразовать df в формат geopandas.

Нужно выяснить сколько было различных проездов автомобиля (прерывания более 0.5 секунды), визуализировать эти маршруты, визуализировать графики скорости проездов.

## Импорт необходимых библиотек

In [1]:
!pip install geopandas

In [15]:
import pandas as pd
import geopandas as gpd
from folium import LayerControl, FeatureGroup
from folium.plugins import AntPath
from pyproj import Transformer
import numpy as np
import seaborn as sns
import os

Настройка глобальных переменных

In [3]:
if 'google.colab' in str(get_ipython()):
    data_dir = os.path.join(os.getcwd(), '/content/data/geopandas')
else:
    data_dir = os.path.join(os.getcwd(), '../../../data/geopandas')

Системы координат

In [4]:
source = "EPSG:32637"
target = "EPSG:4326"

## Решение

Чтение данных

In [5]:
df = pd.read_csv(os.path.join(data_dir, 'geopandas_test.csv'))
df.head()

,Unnamed: 0,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time
0,0,36,ladybug_panoramic_000000,372238.374093,399070.538384,6.186249e+06,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58
1,1,36,ladybug_panoramic_000001,372238.874115,399072.484287,6.186258e+06,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58
2,2,36,ladybug_panoramic_000002,372239.374138,399074.437530,6.186267e+06,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59
3,3,36,ladybug_panoramic_000003,372239.874160,399076.386642,6.186276e+06,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59
4,4,36,ladybug_panoramic_000004,372240.374182,399078.315928,6.186285e+06,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00


Переводим значения координат из системы `EPSG:32637` в систему `EPSG:4326`

In [6]:
transformer = Transformer.from_crs(source, target)
df.X, df.Y, df.Z, df['GPS time'] = transformer.transform(df.X, df.Y, df.Z, df['GPS time'])

Закидываем данные в Geopandas DataFrame

In [7]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y, df.Z, crs=target))
gdf.head()

,Unnamed: 0,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time,geometry
0,0,36,ladybug_panoramic_000000,372238.374093,55.811275,37.389467,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58,POINT Z (55.81128 37.38947 177.28908)
1,1,36,ladybug_panoramic_000001,372238.874115,55.811356,37.389495,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58,POINT Z (55.81136 37.38949 177.25250)
2,2,36,ladybug_panoramic_000002,372239.374138,55.811438,37.389522,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59,POINT Z (55.81144 37.38952 177.21810)
3,3,36,ladybug_panoramic_000003,372239.874160,55.811521,37.389550,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59,POINT Z (55.81152 37.38955 177.17292)
4,4,36,ladybug_panoramic_000004,372240.374182,55.811604,37.389578,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00,POINT Z (55.81160 37.38958 177.12775)


Добавляем дополнительные столбцы:

1. Разница во времени между кадрами
2. Расстояние между 2-мя точками на плоскости
3. Расстояние между 2-мя точками в простанстве
4. Дистанция в метрах между точками
5. Скорость движения от одной точки к другой

Обнуляем эти поля в строках, где разница во времени больше 0,6 секунд, то есть было прерывание

In [8]:
gdf['diff_t'] = gdf['GPS time'].diff().fillna(0)
gdf['diff_v2'] = ((gdf['X'].diff()**2 + gdf['Y'].diff()**2)**(0.5)).fillna(0)
gdf['diff_v3'] = ((gdf['X'].diff()**2 + gdf['Y'].diff()**2 + gdf['Z'].diff()**2)**(0.5)).fillna(0)

points_df = gpd.GeoDataFrame({'geometry': gdf.geometry}, crs='EPSG:4326')
points_df = points_df.to_crs('EPSG:5234')
points_df2 = points_df.shift()
gdf['dist_m'] = points_df.distance(points_df2).fillna(0)
gdf['speed'] = (((gdf['dist_m']/gdf['diff_t'])*3600)/1000).fillna(0)

gdf.loc[(gdf.diff_t > 0.6), ['diff_v2', 'diff_v3', 'dist_m', 'speed']] = 0.0
gdf.head()

,Unnamed: 0,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time,geometry,diff_t,diff_v2,diff_v3,dist_m,speed
0,0,36,ladybug_panoramic_000000,372238.374093,55.811275,37.389467,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58,POINT Z (55.81128 37.38947 177.28908),0.000000,0.000000,0.000000,0.000000,0.000000
1,1,36,ladybug_panoramic_000001,372238.874115,55.811356,37.389495,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58,POINT Z (55.81136 37.38949 177.25250),0.500022,0.000085,0.036583,8.262963,59.490714
2,2,36,ladybug_panoramic_000002,372239.374138,55.811438,37.389522,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59,POINT Z (55.81144 37.38952 177.21810),0.500023,0.000087,0.034399,8.369473,60.257430
3,3,36,ladybug_panoramic_000003,372239.874160,55.811521,37.389550,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59,POINT Z (55.81152 37.38955 177.17292),0.500022,0.000087,0.045181,8.440069,60.765822
4,4,36,ladybug_panoramic_000004,372240.374182,55.811604,37.389578,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00,POINT Z (55.81160 37.38958 177.12775),0.500022,0.000088,0.045171,8.507575,61.251842


In [9]:
gdf.describe()

,Unnamed: 0,panoram stream number,GPS time,X,Y,Z,roll,pitch,heading,diff_t,diff_v2,diff_v3,dist_m,speed
count,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,2.181700e+04,21817.000000,21817.000000,21817.000000
mean,10908.000000,41.221295,379141.252290,55.835970,37.447345,174.761741,176.929788,102.791765,40.719811,0.636480,4.281334e-05,0.042711,4.761175,34.278916
std,6298.169747,3.409455,4220.076046,0.023563,0.047051,13.038282,15.330618,145.081581,99.681990,7.745370,4.025115e-05,0.066685,4.564904,32.865833
min,0.000000,36.000000,372238.374093,55.781439,37.389217,146.231131,0.000323,-179.999371,-179.962409,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,5454.000000,38.000000,375453.291730,55.826173,37.405318,161.312118,177.803379,176.424355,-53.275377,0.500022,4.667378e-07,0.001087,0.052312,0.376627
50%,10908.000000,41.000000,379019.777932,55.832533,37.437783,177.541649,178.397143,178.200587,93.117172,0.500022,3.516986e-05,0.017839,3.782968,27.236115
75%,16362.000000,45.000000,383041.222840,55.856697,37.475279,186.107876,178.842616,178.835657,117.299632,0.500023,7.890722e-05,0.051350,8.351727,60.129787
max,21816.000000,46.000000,386124.463166,55.888866,37.573666,200.233964,179.999877,179.998704,179.890379,636.872752,1.412671e-04,0.607726,16.540548,119.086707


Количество различных проездов автомобиля (прерывания более 0.5 секунды)

In [10]:
len(gdf[gdf['GPS time'].diff() > 0.6])

10

Получаем индексы начала и конца маршрутов (начало и конец определяются по задержки в кадрах)

In [11]:
path_indexes = gdf[(gdf['GPS time'].diff() > 0.6) | (gdf['diff_t'] == 0)].index.values.tolist()
path_indexes.append(gdf.iloc[-1:].index.values[0])
path_indexes

[0, 62, 5335, 7947, 8794, 9270, 11053, 12924, 13384, 16044, 19444, 21816]

Получаем скисок координат точек

In [12]:
coordinates = np.concatenate([np.array([gdf.geometry.x.values]).T, np.array([gdf.geometry.y.values]).T], axis=1) 
coordinates = list(map(list, coordinates))

Список цветов для маршрутов

In [13]:
colors = ['Black', 'Gray', 'Red', 'Green', 'Blue', 'Fuchsia', 'Lime', 'Aqua',
          'Purple', 'Magenta', 'DeepSkyBlue']

Построение карты маршрутов со слоями. Каждый слой отвечает за 1 маршрут. маршруты имеют среднюю скорость на каждые `batch_size` точек

In [18]:
# создаем карту, указывая центр (средние координаты точек) и приближение
m = folium.Map(location=[np.mean(gdf['X'].values), np.mean(gdf['Y'].values)], zoom_start=12)

# количество точек, по которым будет браться средняя скорость
batch_size = 100
# feature_group_all = FeatureGroup(name=f'All (Multi-Color)')

# итерация для каждого маршрута
for path in range(len(path_indexes) - 1):
  # создаем слой
  feature_group_path = FeatureGroup(name=f'Path {path} ({colors[path]})')

  # для текущего маршрута берем скорость и координаты
  path_speeds = gdf['speed'].values.tolist()[path_indexes[path]:path_indexes[path + 1]]
  path_coordinates = coordinates[path_indexes[path]:path_indexes[path + 1]]

  # вспомогательные переменные для работы по отрезкам
  ost = len(path_speeds) % batch_size  
  count = int(len(path_speeds) / batch_size)
  
  # если в маршруте есть больше batch_size точек
  if count > 0:
    # итерация по количеству батчей
    for i in range(count):
      # создаем путь
      ant_path = folium.plugins.AntPath(path_coordinates[i * batch_size:(i + 1) * batch_size], 
                                        f'{np.mean(path_speeds[i * batch_size:(i + 1) * batch_size]):.2f} km/h', 
                                        weight=10, color=colors[path])
      
      # добавляем в слой
      feature_group_path.add_child(ant_path)
    
  # если у маршрута остались точки
  if (ost > 0):
    # добавляем остаточный маршрут в слой
    ant_path = folium.plugins.AntPath(path_coordinates[count * batch_size:count * batch_size + ost], 
                                      f'{np.mean(path_speeds[count * batch_size:count * batch_size + ost]):.2f} km/h', 
                                      weight=10, color=colors[path])#.add_to(feature_group_path)

    feature_group_path.add_child(ant_path)

  # добавляем слой на карту
  feature_group_path.add_to(m)

# добавление манели слоев
LayerControl().add_to(m)

# вывод карты
m